In [ ]:
from hydra import compose, initialize

try:
    initialize(config_path="../configs", job_name="test")
except Exception as e:
    pass

In [ ]:
model_config = compose(config_name="model/faster_rcnn.yaml")

In [ ]:
import pyrootutils

pyrootutils.setup_root(".", indicator=".project-root", pythonpath=True)
from src.models.detection_module import DetectionModule

model = DetectionModule.load_from_checkpoint("../checkpoints/last.ckpt")

In [ ]:
from hydra.utils import instantiate

data_config = compose(
    config_name="data/pascal_voc.yaml", overrides=["+data.val_data.root=../data"]
)
data = instantiate(data_config.data.val_data)

In [ ]:
model.detector.roi_head.test_nms_cfg["nms_thr"] = 0.3

In [ ]:
import torch

x = torch.tensor([0, 1, 2, 2])
expanded = torch.zeros((4, 4))
expanded[range(4), x] = 1
expanded

In [ ]:
import torch

from src.structures import BatchDataSample

model.eval()
data_samples = BatchDataSample([data[8]])
with torch.no_grad():
    det_results = model.predict_step(data_samples)

det_results[0].visualize(backend="matplot")